## Red con Feature Learning
Con esta red se obtuvo un Accuracy en Test de 0.797, el mejor valor obtenido. La red realiza Transfer Learning con EfficientNetB0 y agrega una capa densa a la salida.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rn2021q1itba-cifar100/submission_example.csv
/kaggle/input/rn2021q1itba-cifar100/y_train.npy
/kaggle/input/rn2021q1itba-cifar100/x_test.npy
/kaggle/input/rn2021q1itba-cifar100/x_train.npy


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import keras
import keras.backend as K
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

import albumentations as albu
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

### Parámetros

In [3]:
n_classes = 100
# Parameters for training
epochs = 25
batch_size = 8

### Importación de datos

In [ ]:
X = np.load("/kaggle/input/rn2021q1itba-cifar100/x_train.npy")
X_test = np.load("/kaggle/input/rn2021q1itba-cifar100/x_test.npy")
y = np.load("/kaggle/input/rn2021q1itba-cifar100/y_train.npy")

In [6]:
##### División entre train y validación
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
     X, y, test_size=0.20, random_state=42, stratify=y)

### Red de clasificación
#### Data Augmentation

Implementación de DataGenerator basada en https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

In [9]:
# Función para cambiar el tamaño de la imagen al tamaño necesario utilizando interpolación bicubic.
def np_resize(img, shape):
    return cv2.resize(img, 
        (shape[1], shape[0]), i
        nterpolation = cv2.INTER_CUBIC)

In [10]:

class DataGenerator(keras.utils.Sequence):
    'Generates data for keras'
    def __init__(self, images , labels = None, mode = 'fit', batch_size = batch_size,
                 dim = (224, 224), channels = 3, n_classes = 100,
                 shuffle = True, augment = False):
        self.images = images
        self.labels = labels
        self.mode = mode
        self.batch_size = batch_size
        self.dim = dim
        self.channels = channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augment = augment
        
        self.on_epoch_end()
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.images.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images) / self.batch_size))
        
    def __getitem__(self, index):
        'Generate one batch of data'
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # X
        X = np.empty((self.batch_size, *self.dim, self.channels))
        for i, ID in enumerate(batch_indexes):
            # Generate a preprocessed image
            img = self.images[ID]
            img = img.astype(np.float32) / 255.
            img = np_resize(img, self.dim)
            X[i] = img
            
        # Y
        if self.mode == 'fit':
            y = self.labels[batch_indexes]
            y = to_categorical(y, n_classes)

            if self.augment == True:
                X = self.__augment_batch(X)                
            
            return X,y
        
        elif self.mode == 'predict':
            return X       
            
    def __random_transform(self, img):
        composition = albu.Compose([albu.HorizontalFlip(p = 0.5),
                                    albu.VerticalFlip(p = 0.5),
                                    albu.GridDistortion(p = 0.2),
                                    albu.ElasticTransform(p = 0.2)])
        
        return composition(image = img)['image']
        
    
    def __augment_batch(self, img_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i] = self.__random_transform(img_batch[i])
            
        return img_batch

In [11]:
train_generator = DataGenerator(X_train, y_train, augment = True)
valid_generator = DataGenerator(X_val, y_val, augment = False)

In [12]:
!pip install -U efficientnet

     |████████████████████████████████| 50 kB 520 kB/s eta 0:00:01


In [13]:
import efficientnet.keras as efn 

#Transfer Learning
efnb0 = efn.EfficientNetB0(weights = 'imagenet', include_top = False, classes = n_classes, input_shape = (224,224,3))

model = Sequential()
model.add(efnb0)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(n_classes, activation = 'softmax'))

model.summary()

16809984/16804768 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Functional) (None, 7, 7, 1280)        4049564   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               128100    
Total params: 4,177,664
Trainable params: 4,135,648
Non-trainable params: 42,016
_________________________________________________________________


#### Callbacks

In [14]:
sgd = SGD(lr = 1e-3, momentum = 0.9, nesterov = True)
early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 10, restore_best_weights = True, verbose = 1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', patience = 5, 
                        factor = 0.5, min_lr = 1e-6, verbose = 1)

In [15]:
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['acc'])

#### Entrenamiento

In [19]:
hist = model.fit(train_generator,validation_data = valid_generator, 
                           epochs = epochs, verbose = 1, callbacks = [early_stop, reduce_lr],  steps_per_epoch=len(train_generator))

model.save_weights("best_weight.h5")

Epoch 1/25
625/625 [==============================] - 93s 149ms/step - loss: 1.3492 - acc: 0.6258 - val_loss: 0.9728 - val_acc: 0.7212
Epoch 2/25
625/625 [==============================] - 91s 145ms/step - loss: 1.3624 - acc: 0.6222 - val_loss: 1.0685 - val_acc: 0.6919
Epoch 3/25
625/625 [==============================] - 95s 151ms/step - loss: 1.3454 - acc: 0.6204 - val_loss: 0.9392 - val_acc: 0.7209
Epoch 4/25
625/625 [==============================] - 95s 151ms/step - loss: 1.3234 - acc: 0.6334 - val_loss: 0.8854 - val_acc: 0.7354
Epoch 5/25
625/625 [==============================] - 95s 151ms/step - loss: 1.2998 - acc: 0.6370 - val_loss: 0.8870 - val_acc: 0.7357
Epoch 6/25
625/625 [==============================] - 95s 151ms/step - loss: 1.2952 - acc: 0.6368 - val_loss: 0.8278 - val_acc: 0.7528
Epoch 7/25
625/625 [==============================] - 95s 152ms/step - loss: 1.2885 - acc: 0.6398 - val_loss: 0.9053 - val_acc: 0.7288
Epoch 8/25
625/625 [==============================] - 9

### Test

In [ ]:
# Predicciones
test_generator = DataGenerator(X_test, mode = 'predict', augment = False, shuffle = False)
y_pred = model.predict_generator(test_generator,verbose = 1)
y_pred = np.argmax(y_pred, axis = 1)

In [23]:
y_pred

array([68, 33, 55, ..., 51, 42, 70])

In [25]:
#Salvo las predicciones y genero el archivo csv según el formato pedido
df = pd.DataFrame(y_pred, columns=["label"])
df.index.name = "Id"
df.to_csv("submission2.csv")